In [3]:
//@ts-check

class MyArray {
  /**
   * @param {number[] | boolean[]} flat
   * @param {number[]} shape
   * @param {typeof Number|typeof Boolean} dtype
   */
  constructor(flat, shape, dtype=Number) {
    this.flat = flat;
    this.shape = shape;
    this.dtype = dtype;
    this.positive = undefined;
    this.negative = undefined;
    this.bitwise_not = undefined;
    this.lists = undefined;
    this.isclose = undefined;
    this.allclose = undefined;
    this.grammar = undefined;
    this.nested = undefined;
    this.op = undefined;
  }
  _idx_slice;
  ravel;
  slice;
  reshape;
  from_js_array;
  asarray;
  _binary_operation;
  _reduce;
  sum;
  arange;
  zeros;
  ones;
  _new;
  array;
  to_js_array;
  max;
  min;
  argmin;
  argmax;
  mean;
  var;
  product;
  std;
  add;
  subtract;
  multiply;
  divide;
  pow;
  mod;
  _binary_operations;
  _parse_sliceSpec;
  _parse_sliceRange;
  __parse_sliceSpec;
  __parse_sliceRange;
  __shape_shifts;
  _broadcast_shape;
  _broadcast_shapes;
  __as_bool;
  __parse_list;
  __parse_shape;
  __make_operation;
  any;
  all;
  divide_int;
  __second;
  bitwise_or;
  bitwise_and;
  bitwise_xor;
  bitwise_shift_left;
  bitwise_shift_right;
  gt;
  lt;
  geq;
  leq;
  eq;
  neq;
  or;
  and;
  maximum;
  minimum;
}

In [4]:
//@ts-check
var ohm = require('ohm-js');

var { spawnSync } = require('child_process'); // For testing

// console.log(np`np.arange(120).reshape([-1,3])`)
// console.log(np`np.arange(120).reshape([-1, 3])[2:-3:6]`)



MyArray.prototype._new = function (shape, f, dtype) {
  shape = MyArray.prototype.__parse_shape(shape);
  const size = shape.reduce((a, b) => a * b, 1);
  const flat = Array.from({ length: size }, f);
  return new MyArray(flat, shape, dtype);
};

MyArray.prototype.__parse_shape = function (list) {
  if (Array.isArray(list)) return list;
  if (list instanceof MyArray) {
    if (list.shape.length > 1) {
      throw new Error(`Expected flat list. Got array with shape ${list.shape}`);
    }
    return list.flat;
  }
  if (typeof list == "number") return [list];
  throw new Error(`Expected list. Got ${list}`);
}
MyArray.prototype.zeros = function (shape, dtype = Number) {
  return MyArray.prototype._new(shape, (_) => 0, dtype)
};
MyArray.prototype.ones = function (shape, dtype = Number) {
  return MyArray.prototype._new(shape, (_) => 1, dtype)
};
MyArray.prototype.arange = function (arg0, arg1) {
  let start, end;
  if (arg1 === undefined) start = 0, end = arg0;
  else start = arg0, end = arg1;
  return MyArray.prototype._new(end - start, (_, i) => start + i, Number)
};

// ==============================
//       Reducing functions
// ==============================


MyArray.prototype._reduce = function (arr, axis, keepdims, reducer, dtype = Number) {
  keepdims = MyArray.prototype.__as_bool(keepdims);
  arr = MyArray.prototype.asarray(arr);
  if (axis == null) return reducer(arr.flat);
  if (axis < 0) axis = arr.shape.length - 1;
  let m = arr.shape[axis];
  let shift = MyArray.prototype.__shape_shifts(arr.shape)[axis];
  const groups = Array.from({ length: m }, (_) =>/**@type {number[]}*/([]));
  arr.flat.forEach((value, i) => groups[(Math.floor(i / shift)) % m].push(value));
  // Transpose it:
  let nCols = arr.flat.length / m;
  const groupsT = [];
  for (let j = 0; j < nCols; j++) {
    const newRow = [];
    for (let i = 0; i < m; i++) newRow.push(groups[i][j]);
    groupsT.push(newRow);
  }
  const flat = groupsT.map(reducer);
  let shape = [...arr.shape];
  if (keepdims) shape[axis] = 1;
  else shape = shape.filter((_, i) => i != axis);
  return new MyArray(flat, shape, dtype);
};
MyArray.prototype.__as_bool = function (obj) {
  if (typeof obj == 'string') throw new Error(`'string' object can not be interpreted as boolean: ${obj}`);
  return !!(0 + obj);
}

MyArray.prototype.any = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a || b, false), Boolean);
};
MyArray.prototype.all = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a && b, true), Boolean);
};
MyArray.prototype.sum = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a + b, 0));
};
MyArray.prototype.product = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a * b, 1));
};
MyArray.prototype.max = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => Math.max(...arr));
};
MyArray.prototype.min = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => Math.min(...arr));
};
MyArray.prototype.argmax = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.indexOf(Math.max(...arr)));
};
MyArray.prototype.argmin = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.indexOf(Math.min(...arr)));
};
MyArray.prototype.mean = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  return MyArray.prototype._reduce(arr, axis, keepdims, (arr) => arr.reduce((a, b) => a + b, 0) / arr.length);
};
MyArray.prototype.var = function (arr) {
  const { axis, keepdims } = Object.assign({ axis: null, keepdims: false }, this);
  const mean = MyArray.prototype.mean.bind({ axis, keepdims: true })(arr);
  arr = MyArray.prototype.subtract(arr, mean);
  arr = MyArray.prototype.multiply(arr, arr);
  return MyArray.prototype.mean.bind({ axis, keepdims })(arr);
};
MyArray.prototype.std = function (arr) {
  const variance = MyArray.prototype.var.bind(this)(arr);
  return MyArray.prototype.pow(variance, 0.5);
};


// ==============================
//       Binary operations
// ==============================

MyArray.prototype._binary_operations = {
  "+": [((a, b) => a + b), Number],
  "-": [((a, b) => a - b), Number],
  "*": [((a, b) => a * b), Number],
  "/": [((a, b) => a / b), Number],
  "%": [((a, b) => (a % b)), Number],
  "//": [((a, b) => Math.floor(a / b)), Number],
  "**": [((a, b) => Math.pow(a, b)), Number],
  "=": [((a, b) => b), Number],
  "|": [((a, b) => a | b), Number],
  "&": [((a, b) => a & b), Number],
  "^": [((a, b) => a ^ b), Number],
  "<<": [((a, b) => a << b), Number],
  ">>": [((a, b) => a >> b), Number],
  ">": [((a, b) => a > b), Boolean],
  "<": [((a, b) => a < b), Boolean],
  ">=": [((a, b) => a >= b), Boolean],
  "<=": [((a, b) => a <= b), Boolean],
  "==": [((a, b) => a == b), Boolean],
  "!=": [((a, b) => a != b), Boolean],
  "||": [((a, b) => a || b), Boolean],
  "&&": [((a, b) => a && b), Boolean],
  // Custom notation
  "↑": [((a, b) => Math.max(a, b)), Number],
  "↓": [((a, b) => Math.min(a, b)), Number],
  // "≈": [((a, b) => a, b), Boolean],
};

MyArray.prototype._binary_operation = function (A, B, func, dtype) {
  // Find output shape and input broadcast shapes
  A = MyArray.prototype.asarray(A);
  B = MyArray.prototype.asarray(B);
  const [shape, shapeA, shapeB] = MyArray.prototype._broadcast_shapes(A.shape, B.shape);
  const shiftsA = MyArray.prototype.__shape_shifts(shapeA);
  const shiftsB = MyArray.prototype.__shape_shifts(shapeB);
  // Iterate with broadcasted indices
  const result = new Array(shape.reduce((a, b) => a * b, 1)).fill(undefined);
  for (let i = 0; i < result.length; i++) {
    let idxA = 0, idxB = 0, idx = i;
    for (let axis = shape.length - 1; axis >= 0; axis--) {
      idxA += shiftsA[axis] * (idx % shapeA[axis]);
      idxB += shiftsB[axis] * (idx % shapeB[axis]);
      idx = Math.floor(idx / shape[axis]);
    }
    result[i] = func(A.flat[idxA], B.flat[idxB]);
  };
  return new MyArray(result, shape, dtype);
}

MyArray.prototype.__make_operation = function (symbol) {
  const [func, dtype] = MyArray.prototype._binary_operations[symbol];
  return function (A, B) {
    return MyArray.prototype._binary_operation(A, B, func, dtype);
  };
}

MyArray.prototype._broadcast_shapes = function (shapeA, shapeB) {
  const shape = [];
  const maxDim = Math.max(shapeA.length, shapeB.length);
  shapeA = [...Array.from({ length: maxDim - shapeA.length }, () => 1), ...shapeA];
  shapeB = [...Array.from({ length: maxDim - shapeB.length }, () => 1), ...shapeB];
  for (let axis = maxDim - 1; axis >= 0; axis--) {
    const dim1 = shapeA[axis];
    const dim2 = shapeB[axis];
    if (dim1 !== 1 && dim2 !== 1 && dim1 !== dim2)
      throw new Error(`Can not broadcast axis ${axis} with sizes ${dim1} and ${dim2}`);
    shape.unshift(Math.max(dim1, dim2));
  }
  return [shape, shapeA, shapeB];
}

MyArray.prototype.add = MyArray.prototype.__make_operation("+");
MyArray.prototype.subtract = MyArray.prototype.__make_operation("-");
MyArray.prototype.multiply = MyArray.prototype.__make_operation("*");
MyArray.prototype.divide = MyArray.prototype.__make_operation("/");
MyArray.prototype.mod = MyArray.prototype.__make_operation("%");
MyArray.prototype.divide_int = MyArray.prototype.__make_operation("//");
MyArray.prototype.pow = MyArray.prototype.__make_operation("**");
MyArray.prototype.bitwise_or = MyArray.prototype.__make_operation("|");
MyArray.prototype.bitwise_and = MyArray.prototype.__make_operation("&");
MyArray.prototype.bitwise_xor = MyArray.prototype.__make_operation("^");
MyArray.prototype.bitwise_shift_left = MyArray.prototype.__make_operation("<<");
MyArray.prototype.bitwise_shift_right = MyArray.prototype.__make_operation(">>");
MyArray.prototype.gt = MyArray.prototype.__make_operation(">");
MyArray.prototype.lt = MyArray.prototype.__make_operation("<");
MyArray.prototype.geq = MyArray.prototype.__make_operation(">=");
MyArray.prototype.leq = MyArray.prototype.__make_operation("<=");
MyArray.prototype.eq = MyArray.prototype.__make_operation("==");
MyArray.prototype.neq = MyArray.prototype.__make_operation("!=");
MyArray.prototype.maximum = MyArray.prototype.__make_operation("↑");
MyArray.prototype.minimum = MyArray.prototype.__make_operation("↓");
MyArray.prototype.__second = MyArray.prototype.__make_operation("=");

// Unary operations. Only bitwise_not is included. Positive is useless and negative is almost useless
MyArray.prototype.bitwise_not = function (A) { return MyArray.prototype.bitwise_xor(A, 1); };


MyArray.prototype.op = {
  "+": MyArray.prototype.add,
  "-": MyArray.prototype.subtract,
  "*": MyArray.prototype.multiply,
  "/": MyArray.prototype.divide,
  "%": MyArray.prototype.mod,
  "//": MyArray.prototype.divide_int,
  "**": MyArray.prototype.pow,
  "|": MyArray.prototype.bitwise_or,
  "&": MyArray.prototype.bitwise_and,
  "^": MyArray.prototype.bitwise_xor,
  "<<": MyArray.prototype.bitwise_shift_left,
  ">>": MyArray.prototype.bitwise_shift_right,
  "<": MyArray.prototype.lt,
  ">": MyArray.prototype.gt,
  "<=": MyArray.prototype.leq,
  ">=": MyArray.prototype.geq,
  "==": MyArray.prototype.eq,
  "!=": MyArray.prototype.neq,
  "~": MyArray.prototype.bitwise_not,
  // Operators with custom identifiers:
  "≈≈": MyArray.prototype.isclose,
  "↑": MyArray.prototype.maximum,
  "↓": MyArray.prototype.minimum,
  "≤": MyArray.prototype.leq,
  "≥": MyArray.prototype.geq,
  "≠": MyArray.prototype.neq,
  "=": MyArray.prototype.__second,
}


MyArray.prototype.asarray = function (A) {
  if (A instanceof MyArray) return A;
  else return MyArray.prototype.from_js_array(A);
}
MyArray.prototype.array = function (A) {
  // @ts-ignore
  if (A instanceof MyArray) return new MyArray([...A.flat], [...A.shape], A.dtype);
  else return MyArray.prototype.from_js_array(A);
}
MyArray.prototype.from_js_array = function (arr) {
  if (typeof arr === "number") return new MyArray([arr], [], Number);
  if (typeof arr === "boolean") return new MyArray([arr ? 1 : 0], [], Boolean);
  if (!Array.isArray(arr)) throw new Error(`Can't parse as array: ${arr}`);
  const shape = [];
  let root = arr;
  while (Array.isArray(root)) {
    shape.push(root.length);
    root = root[0];
    if (shape.length > 256) throw new Error(`Circular reference or excessive array depth`);
  }
  let dtype = typeof root === "boolean" ? Boolean : Number;
  const flat = [];
  const pushToFlat = (arr, axis) => {
    // Check consistency
    if (axis == shape.length - 1) {
      for (let elem of arr) {
        if (Array.isArray(elem)) throw new Error(`Inconsistent shape`);
        flat.push(elem);
        // Update dtype
      }
    } else {
      if (!Array.isArray(arr)) throw new Error(`Inconsistent shape`);
      for (let sub of arr) {
        if (sub.length != shape[axis + 1])
          throw new Error(`Inconsistent shape: found sibling arrays of lengths ${sub.length} and ${shape[axis + 1]}`);
        pushToFlat(sub, axis + 1);
      }
    }
  }
  pushToFlat(arr, 0);
  return new MyArray(flat, shape, dtype)
}
MyArray.prototype.to_js_array = function (arr) {
  if (!(arr instanceof MyArray)) throw new Error(`Expected MyArray: ${arr}`);
  if (arr.shape.length == 0) return arr.flat[0];
  function recursiveReshape(flatArr, shapeArr) {
    if (shapeArr.length === 0) {
      return flatArr.shift();
    }
    const innerShape = shapeArr.slice(1);
    const outerSize = shapeArr[0];
    const innerArray = [];
    for (let i = 0; i < outerSize; i++) {
      innerArray.push(recursiveReshape(flatArr, innerShape));
    }
    return innerArray;
  }
  return recursiveReshape(arr.flat, arr.shape);
}
MyArray.prototype.ravel = function (A) {
  A = MyArray.prototype.asarray(A);
  return new MyArray([...A.flat], [A.flat.length], A.dtype);
};

// =========================================
//     Slicing
// =========================================


MyArray.prototype.slice = function (arr, sliceSpec) {
  const [shape, indices] = MyArray.prototype._idx_slice(arr.shape, sliceSpec);
  return new MyArray(indices.map(i => arr.flat[i]), shape);
}

MyArray.prototype.__shape_shifts = function (shape) {
  // increasing one by one on a given axis is increasing by shifts[axis] in flat representation
  const shifts = Array.from({ length: shape.length }, (_) => 0);
  shifts[shape.length - 1] = 1;
  for (let i = shape.length - 2; i >= 0; i--) shifts[i] = shifts[i + 1] * shape[i + 1];
  return shifts;
}
MyArray.prototype.__parse_sliceRange = function (axis_size, { start, stop, step }) {
  if (start == null) start = 0;
  else if (start < 0) start = axis_size + start;
  if (stop == null) stop = axis_size;
  else if (stop < 0) stop = axis_size + stop;
  if (step == null) step = 1;
  else if (step == 0) throw new Error(`Slice range with step size of zero`);
  if (!isFinite(start) || !isFinite(stop) || !isFinite(step)) throw new Error(`Invalid slice ${[start, stop, step]}. Axis size ${axis_size}`);
  let indices = [];
  if (step > 0) {
    start = Math.max(start, 0);
    stop = Math.min(stop, axis_size);
    for (let i = start; i < stop; i += step) indices.push(i);
  } else {
    stop = Math.max(stop, 0);
    start = Math.min(start, axis_size);
    for (let i = start; i > stop; i += step) indices.push(i);
  }
  return indices;
}



MyArray.prototype.__parse_sliceSpec = function (shape, sliceSpec) {
  /**
   * 
   * virtualShape (hard to explain, but equal to shape most of the times).
   * virtualShape is used to capture the fact that a boolean mask can span
   * over several axes. E.g., in np.ones((2,3,4,5))[:, np.random.random((3,4))>0.5, 1],
   * the random boolean mask is spanning over axes 1 and 2. In this case, it suffices
   * to merge these axes, to get a virtualShape of (2, 12, 5), and flattening the mask.
   */
  const virtualShape = [];
  const outShape = [];
  const slices = [];
  let maxI = Math.max(shape.length, sliceSpec.length);
  for (let i = 0, axis = 0; i < maxI; i++) {
    let slice = sliceSpec[i];
    if (slice == "None") {
      outShape.push(1);
      continue;
    }
    if (slice === undefined) slice = ":";
    else if (Array.isArray(slice)) slice = MyArray.prototype.array(slice);
    let indices, collapse = false, nextAxis = axis + 1;
    if (slice == ':') indices = Array.from({ length: shape[axis] }, (_, i) => i);
    else if (slice === parseInt(slice)) collapse = true, indices = [slice];
    else if (slice.isRange) indices = MyArray.prototype.__parse_sliceRange(shape[axis], slice);
    else if (slice instanceof MyArray) {
      if (slice.dtype == Number) { // Array of indices
        if (slice.shape.length > 1) throw new Error(
          `Expected 1D array of indices or nD array of booleans. ` +
          `Found shape=${slice.shape} and dtype=${slice.dtype}`
        );
        indices = slice.flat;
      } else { // binary mask
        indices = [];
        slice.flat.forEach((if_value, i) => if_value && indices.push(i));
        // Next lines: special (not so common) case: the boolean mask spans over more than 1 axis
        nextAxis = axis + Math.max(1, slice.shape.length);
      }
    } else throw new Error(`Expected array. Found ${typeof slice}: ${slice}`);
    slices.push(indices);
    if (!collapse) outShape.push(indices.length);
    virtualShape.push(nextAxis == axis + 1 ? shape[axis] : shape.slice(axis, nextAxis).reduce((a, b) => a * b, 1));
    axis = nextAxis;
  }
  return [outShape, slices, virtualShape];
}


MyArray.prototype._idx_slice = function (shape, sliceSpec) {
  // Iterative cartesian product of the slices.
  const [outShape, slices, vShape] = MyArray.prototype.__parse_sliceSpec(shape, sliceSpec);
  if (slices.map(l => l.length).reduce((a, b) => a * b, 1) == 0) {
    return [outShape, []];
  }
  const shifts = MyArray.prototype.__shape_shifts(vShape);
  const indices = [];
  const maxIndex = slices.length - 1;
  const tuple = new Array(slices.length).fill(0);
  let current = shifts.map((v, i) => v * slices[i][0]).reduce((a, b) => a + b, 0);
  while (true) {
    // const currentTuple = tuple.map((index, arrayIndex) => slices[arrayIndex][index]);
    indices.push(current);
    // Increment the rightmost index
    let i = ++tuple[maxIndex];
    if (i < slices[maxIndex].length) {
      current += (slices[maxIndex][i] - slices[maxIndex][i - 1]) * shifts[maxIndex];
    }
    // Check for overflow in each dimension
    for (let i = maxIndex; i > 0; i--) {
      if (tuple[i] === slices[i].length) {
        tuple[i] = 0;
        let j = ++tuple[i - 1];
        current -= (slices[i][slices[i].length - 1] - slices[i][0]) * shifts[i];
        if (j < slices[i - 1].length) current += (slices[i - 1][j] - slices[i - 1][j - 1]) * shifts[i - 1];
      }
    }
    if (!isFinite(current)) throw `Programming error`
    // Check if we have finished iterating
    if (tuple[0] === slices[0].length) break;
  }
  return [outShape, indices];
}

MyArray.prototype.isclose = function (A, B, rtol = 1.e-5, atol = 1.e-8, equal_nan = false) {
  const func = (a, b) => {
    if (Number.isFinite(a) && Number.isFinite(b)) return Math.abs(a - b) <= atol + rtol * b;
    return (a == b) || (equal_nan && Number.isNaN(a) && Number.isNaN(b));
  }
  return MyArray.prototype._binary_operation(A, B, func, Boolean)
}

MyArray.prototype.allclose = function (A, B, rtol = 1.e-5, atol = 1.e-8, equal_nan = false) {
  // Equivalent to all(isclose(A, B, rtol, atol, equal_nan)), but shortcutting if false 
  const func = (a, b) => { //copied from isclose
    if (Number.isFinite(a) && Number.isFinite(b)) return Math.abs(a - b) <= atol + rtol * b;
    return (a == b) || (equal_nan && Number.isNaN(a) && Number.isNaN(b));
  }
  const different = new Error('');
  const wrapper = (a, b) => {
    if (!func(a, b)) throw different;
    return 0;
  }
  try { MyArray.prototype._binary_operation(A, B, wrapper, Number) }
  catch (err) {
    if (err === different) return false;
    else throw err;
  }
  return true;
}


MyArray.prototype.reshape = function (A, shape) {
  shape = [...shape];
  const n = A.flat.length;
  const inferredIndex = shape.indexOf(-1);
  if (inferredIndex !== -1) {
    const productOfKnownDims = shape.filter(dim => dim !== -1).reduce((acc, val) => acc * val, 1);
    if (n % productOfKnownDims !== 0) {
      throw new Error("Invalid shape. The total number of elements must match the product of the known dimensions.");
    }
    shape[inferredIndex] = n / productOfKnownDims;
  }
  return new MyArray([...A.flat], shape);
};


MyArray.prototype.nested = {
  _binary_operation(A, B, func) {
    // Pointwise check for equality for arbitrary nested js arrays (without broadcasting)
    const C = [];
    const q = [[A, B, C, 0]];
    let seen;
    while (true) {
      const _next = q.pop();
      if (!_next) return true;
      const [a, b, c, depth] = _next
      if (Array.isArray(a) && Array.isArray(b) && a.length == b.length) {
        for (let i in a) {
          const c_i = [];
          c.push(c_i);
          q.push([a[i], b[i], c_i, depth + 1]);
        }
      }
      else c.push(func(a, b));
      if (depth > 10000) { // Activate circular reference detection
        // Checking only A suffices (the other will exhaust otherwise)
        seen = /**@type {any[]}*/(seen || []);
        if (seen.includes(a)) throw new Error(`Circular reference found. ${a}`)
        seen.push(a);
      }
    }
  },
  ravel(A) {
    // Flatten js array
    const q = [[A, 0]], flat = [];
    while (true) {
      const _next = q.pop();
      if (!_next) break;
      const [a, depth] = _next
      if (Array.isArray(a)) {
        q.push(...a.map(v => [v, depth + 1]));
        continue;
      }
      flat.push(a);
      if (depth > 10000) { // Activate circular reference detection
        if (flat.includes(a)) throw new Error(`Circular reference found. ${a}`);
      }
    }
    return flat;
  },
  allEq(A, B, nan_equal = false) {
    const different = new Error('');
    const func = (a, b) => {
      if (a !== b && !(nan_equal && Number.isNaN(a) && Number.isNaN(b))) throw different;
      return 0;
    }
    try { MyArray.prototype.nested._binary_operation(A, B, func, Number) }
    catch (err) {
      if (err === different) return false;
      else throw err;
    }
    return true;
  },
  allClose(A, B, rtol = 1.e-5, atol = 1.e-8, nan_equal = false,) {
    const func = (a, b) => { //copied from isclose
      if (Number.isFinite(a) && Number.isFinite(b)) return Math.abs(a - b) <= atol + rtol * Math.abs(b);
      return (a === b) || (nan_equal && Number.isNaN(a) && Number.isNaN(b));
    }
    const different = new Error('');
    const wrapper = (a, b) => {
      if (!func(a, b)) throw different;
      return 0;
    }
    try { MyArray.prototype.nested._binary_operation(A, B, wrapper, Number) }
    catch (err) {
      if (err === different) return false;
      else throw err;
    }
    return true;
  },
}



MyArray.prototype.grammar = {}
MyArray.prototype.grammar.grammar = String.raw`
ArrayGrammar {
  Instruction
  = Variable "[" Slice "]" AssignSymbol ArithmeticLogicExp -- sliceAssignment
  | ArithmeticLogicExp                       -- expression
  
  Variable
   = "#" digit+ "#"
  
  AssignSymbol
  ="="|"+="|"-="|"/="|"%="|"&="|"|="|"^="|"@="
  
  /* Declaration in precedence order (weakest first) */
  ArithmeticLogicExp = Precedence11

  /* https://docs.python.org/3/reference/expressions.html */
  Operator11 = "<" | "<=" | ">" | ">=" | "!=" | "=="
  Operator10 = "|"
  Operator09 = "^"
  Operator08 = "&"
  Operator07 = "<<" | ">>"
  Operator06 = "+" | "-"
  Operator05 = "*" | "@" | "/" | "//" | "%"
  Operator04 = "~" /* Unary */
  Operator03 = "+" | "-" /* Unary. Special treatment to prevent "-1.3" to be "-(array of 1.3)" */
  Operator02 = "**"
  /* Operator01 = "x[index]" | "x[index:index]" | "x(arguments...)" | "x.attribute" */
  /* Operator00 = "(expressions...)" */

  Precedence11 = Precedence11 Operator11 Precedence10 | "" "" Precedence10
  Precedence10 = Precedence10 Operator10 Precedence09 | "" "" Precedence09
  Precedence09 = Precedence09 Operator09 Precedence08 | "" "" Precedence08
  Precedence08 = Precedence08 Operator08 Precedence07 | "" "" Precedence07
  Precedence07 = Precedence07 Operator07 Precedence06 | "" "" Precedence06
  Precedence06 = Precedence06 Operator06 Precedence05 | "" "" Precedence05
  Precedence05 = Precedence05 Operator05 Precedence04 | "" "" Precedence04
  Precedence04 = ""           Operator04 Precedence03 | "" "" Precedence03 /* Unary */
  Precedence03 = ""           Operator03 Precedence02 | "" "" Precedence02 /* Special */
  Precedence02 = Precedence02 Operator02 Precedence03 | "" "" Precedence01
  Precedence01 = Arr
  
  Parenthesis = "(" ArithmeticLogicExp ")"
  Arr
    = Arr "." Name CallArgs     -- method
    | Arr "." Name              -- attribute
    | Arr "[" Slice "]"         -- slice
    | Parenthesis
    | Name ("." Name)* CallArgs -- call
    | number
    | Variable

  Name  (an identifier)
    = (letter|"_") (letter|"_"|digit)*

  number  (a number)
    = ("+"|"-")? digit* "." digit+ "E" ("+"|"-")? "digit+"
    | ("+"|"-")? digit* "." digit+ "e" ("+"|"-")? "digit+"
    | ("+"|"-")? digit* "." digit+  ""  ""        ""
    | ("+"|"-")? digit+ ""  ""      ""  ""        ""
  
  int (an integer) = ""  digit+ | "-" digit+ | "+" digit+

  CallArgs // Using empty strings instead of separate rules
   = "(" Args ","  KwArgs ","? ")"
   | "(" Args ","? ""      ""  ")"
   | "(" ""   ","? KwArgs ","? ")"
   | "(" ""    ""  ""      ""  ")"
   
  Args = NonemptyListOf<ArgValue, ",">
  KwArgs = NonemptyListOf<KwArg, ",">
  KwArg = Name "=" ArgValue

  ArgValue = Constant | JsArray | ArithmeticLogicExp
  Constant = "True" | "False" | "None" | "np.nan" | "np.inf" | String
  JsArray = "[" ListOf<ArgValue, ","> ","? "]"

  String = "\'" any* "\'" | "\"" any* "\""
   
  Slice = NonemptyListOf<SliceTerm, ",">
  SliceTerm
    = SliceRange
    | (":" | "..." | "None") -- constant
    | JsArray
    | ArithmeticLogicExp
  
  SliceRange
    = int ":" int ":" int
    | int ":" int ""  ""
    | int ":" ""  ":" int
    | int ":" ""  ""  ""
    | ""  ":" int ":" int
    | ""  ":" int ""  ""
    | ""  ":" ""  ":" int
    | ""  ":" ""  ""  ""
}
`;

MyArray.prototype.grammar.ohmGrammar = ohm.grammar(MyArray.prototype.grammar.grammar);


MyArray.prototype.grammar.__makeSemantics = () => {

  const semanticVariables = [];
  const semantics = {
    Instruction_sliceAssignment($tgt, _open, $sliceSpec, _close, $symbol, $src) {
      // WARNING: Creates a copy. This is terrible for arr[2, 4, 3] = 5
      const tgt = $tgt.parse();
      const symbol = $symbol.sourceString;
      const sliceSpec = $sliceSpec.parse();
      const src = MyArray.prototype.asarray($src.parse());
      let arr = MyArray.prototype.asarray(tgt);
      let [shape, indices] = MyArray.prototype._idx_slice(arr.shape, sliceSpec);
      let result = new MyArray(indices.map(i => arr.flat[i]), shape);
      result = MyArray.prototype._binary_operation(result, src, symbol);
      if (indices.length != result.flat.length) throw new Error(`Can't assign size ${result.flat.length} to slice of size ${indices.length}`);
      for (let i of indices) arr.flat[i] = result.flat[i];
      arr = MyArray.prototype.to_js_array(arr);
      while (tgt.length) tgt.pop();
      tgt.push(...arr);
      return null;
    },
    Instruction_expression($arr) {
      const arr = $arr.parse();
      if (typeof arr === "number") return arr;
      if (Array.isArray(arr)) return arr;
      return MyArray.prototype.to_js_array(arr);
    },
    Precedence11: BinaryOperation,
    Precedence10: BinaryOperation,
    Precedence09: BinaryOperation,
    Precedence08: BinaryOperation,
    Precedence07: BinaryOperation,
    Precedence06: BinaryOperation,
    Precedence05: BinaryOperation,
    Precedence04: UnaryOperation,
    Precedence03: UnaryOperation,
    Precedence02: BinaryOperation,
    number: function (arg1, arg2, arg3, arg4, arg5, arg6, arg7) {
      return parseFloat(this.sourceString)
    },
    Arr_slice($arr, _open, $sliceSpec, _close) {
      const arr = $arr.parse();
      const sliceSpec = $sliceSpec.parse();
      return MyArray.prototype.slice(arr, sliceSpec);
    },
    SliceTerm_constant($x) {
      return $x.sourceString;
    },
    Arr_call($name, $names, _, $callArgs) {
      let name = $name.sourceString + $names.sourceString;
      if (name.slice(0, 3) == "np.") name = name.slice(3);
      const func = MyArray.prototype[name];
      if (func === undefined) throw new Error(`Unrecognized function ${name}`)
      const { args, kwArgs } = $callArgs.parse();
      return func.bind(kwArgs)(...args);
    },
    Arr_method($arr, _dot, $name, $callArgs) {
      let arr = $arr.parse();
      let name = $name.sourceString;
      if (name.slice(0, 3) == "np.") name = name.slice(3);
      const func = MyArray.prototype[name];
      if (func === undefined) throw new Error(`Unrecognized method ${name}`)
      const { args, kwArgs } = $callArgs.parse();
      return func.bind(kwArgs)(arr, ...args);
    },
    Parenthesis(_, $arr, __) { return $arr.parse(); },
    Arr_attribute($arr, _, $name) { return $arr.parse()[$name.sourceString]; },
    Variable(_, $i, __) {
      const i = parseInt($i.sourceString);
      const arr = semanticVariables[i];
      return arr;
    },
    int($sign, $value) {
      const value = parseInt($value.sourceString);
      if ($sign.sourceString == '-') return -value;
      else return value
    },
    SliceRange($start, _, $stop, __, $step) {
      const start = $start.parse();
      const stop = $stop.parse();
      const step = $step.parse();
      return { start, stop, step, isRange: true };
    },

    CallArgs(_open, $args, _comma, $kwArgs, _trailing, _close) {
      const args = $args.parse() || [];
      let parse_constants = (s) => {
        switch (s) {
          case "True": return true;
          case "False": return false;
          case "None": return null;
        }
        if (s.length && s[0] == '"' && s[s.length - 1] == '"') return s;
        if (s.length && s[0] == "'" && s[s.length - 1] == "'") return s;
        return s;
      }
      let entries = $kwArgs.parse() || [];
      entries = entries.map(([k, v]) => [k, parse_constants(v)]);
      let kwArgs = Object.fromEntries(entries);
      return { args, kwArgs };
    },
    KwArg($key, _equals, $value) {
      const key = $key.sourceString;
      const value = $value.sourceString;
      return [key, value];
    },
    NonemptyListOf(first, _, more) {
      return [first, ...more.children].map(c => c.parse());
    },
    JsArray(_open, $list, _trailing, _close) {
      const list = $list.parse();
      // Downcast arrays (needed because, e.g., for [-1, 3, -2], -1 and -2 are interpreted as MyArray rather than int)
      for (let i in list) if (list[i] instanceof MyArray) list[i] = MyArray.prototype.to_js_array(list[i]);
      return list;
    },
    _terminal() { return null; },
  };

  function BinaryOperation($A, $symbol, $B) {
    const A = $A.parse();
    const B = $B.parse();
    const symbol = $symbol.sourceString;
    if (symbol == "" && A === null) return B;
    const [func, dtype] = MyArray.prototype._binary_operations[symbol];
    return MyArray.prototype._binary_operation(A, B, func, dtype);
  }
  function UnaryOperation(_, $symbol, $B) {
    const B = $B.parse();
    const symbol = $symbol.sourceString;
    if (symbol == "") return B;
    switch (symbol) {
      case "+": return B;
      case "-": return MyArray.prototype.multiply(-1, B);
      case "^": return MyArray.prototype.bitwise_not(B);
    }
    throw new Error(`Programming Error: ${symbol}`);
  }

  const { ohmGrammar } = MyArray.prototype.grammar;

  const ohmSemantics = ohmGrammar.createSemantics();
  ohmSemantics.addOperation('parse', semantics);
  /**
   * @param {TemplateStringsArray} template
   * @param {any[]} variables
   */
  function parse(template, ...variables) {
    // Replace variables in template with #0# #1# #2#, ...
    let idx = 0;
    const template_with_placeholders = template.join('###').replace(/###/g, () => `#${idx++}#`);
    semanticVariables.length = 0;
    semanticVariables.push(...variables);
    const match = ohmGrammar.match(template_with_placeholders);
    if (!match.succeeded()) throw new Error(match.message);
    return ohmSemantics(match).parse();
  }

  return { parse, ohmSemantics, semantics, semanticVariables, busy: false };
}


MyArray.prototype.grammar.__parser_pool = [MyArray.prototype.grammar.__makeSemantics()];

/**
 * @param {TemplateStringsArray} template
 * @param {any[]} variables
 */
MyArray.prototype.grammar.parse = function(template, ...variables) {
  // Thread control, because the parser depends on semanticVariables,
  // but we don't want to waste CPU time recreating the parser on each call
  // No cleaning is done (we assume that the number of threads is negligible compared to the memory size)
  const { __parser_pool: pool } = MyArray.prototype.grammar;
  for (let i = 0; i < pool.length; i++) {
    const parser = pool[i];
    if (!parser.busy) {
      parser.busy = true;
      try { return parser.parse(template, ...variables); }
      finally { parser.busy = false; }
    }
    if (i == pool.length) pool.push(MyArray.prototype.grammar.__makeSemantics());
  }
}


function npTest(template, ...variables) {
  let idx = 0;
  const str = template.join('###').replace(/###/g, () => {
    let value = variables[idx++];
    let out = JSON.stringify(value)
    if (Array.isArray(value)) out = `np.array(${out})`;
    return out;
  });
  const program = `
import numpy as np
import json
out = ${str}
if isinstance(out, np.ndarray):
    out = out.tolist()
print(json.dumps(out), flush=True)
`
  const process = spawnSync('python3', ['-c', program]);
  const stdout = process.stdout.toString();
  if (!stdout.length) throw new Error(process.stderr.toString());
  const expected = JSON.parse(stdout);
  let obtained
  try {
    obtained = np(template, ...variables);
  } catch (err) {
    console.error('ERROR. Mismatch for', str);
    console.error(JSON.stringify(expected));
    throw err;
  }
  if (!MyArray.prototype.nested.allClose(obtained, expected)) {
    console.error('ERROR. Mismatch for', str);
    console.error(JSON.stringify(expected));
    console.error(JSON.stringify(obtained));
  }
  return obtained;
}



/**
 * @param {TemplateStringsArray} template
 * @param {any[]} variables
 * */
function np(template, ...variables) {
  return MyArray.prototype.grammar.parse(template, ...variables);
}


MyArray.prototype.linspace = function (start, stop, num = 50, endpoint = true, retstep = false) {
  const { multiply, arange, add } = MyArray.prototype;
  stop = parseFloat(stop);
  start = parseFloat(start);
  let n = (num + (endpoint ? 0 : 1))
  let arr = add(multiply(arange(num), 1 / (stop - start) / n), start);
  return arr;
}

np`linspace(0, 1, 10)`

np`linspace(0, 1, 10)`

[
  0,
  0.1,
  0.2,
  0.30000000000000004,
  0.4,
  0.5,
  0.6000000000000001,
  0.7000000000000001,
  0.8,
  0.9
]

In [5]:

// Unit tests:


npTest`-1**2`
npTest`2**-0.5`
npTest`2**-0.5**2`
npTest`2**-0.5*2`
npTest`2**-(0.5)`

npTest`-1**2`
npTest`2**-0.5`
npTest`2**-0.5**2`
npTest`2**-0.5*2`
npTest`2**-(0.5)`

npTest`np.arange(120).reshape([-1,3])`
npTest`np.zeros(3)`
npTest`np.zeros(${[3, 5, 4]})`
npTest`np.sum(np.ones(${[10, 2]}), axis=0, keepdims=True) + np.arange(2)`
npTest`np.ravel(np.reshape( np.arange(120), ${[2, 3, 4, 5]} ))`
npTest`np.ravel(np.reshape( np.arange(120), ${[2, 3, 4, 5]} )[ :, 0, ${[1, 2]}, : ])`
npTest`np.ravel([[1,2], [3, 4]])`
npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],None,:].shape`
npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],:].shape`
npTest`np.arange(120).std()`

var a = npTest`np.array([[1,2], [3,6], [9, 10]])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
var a = npTest`np.array([[1,2], [2,4], [3,6]])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
npTest`${a}.sum(axis=0, keepdims=True)`
npTest`${a}.sum(axis=1, keepdims=True)`
npTest`${a}.sum(axis=0, keepdims=False)`

var a = npTest`np.arange(12).reshape([2, 3, 2])`
npTest`${a}.sum(axis=0)`
npTest`${a}.sum(axis=1)`
npTest`${a}.sum(axis=-1)`

var a = npTest`np.arange(6).reshape([2, 3])`
npTest`${a} * ${a}`
npTest`2 * ${a}`
npTest`-(${a} * ${a})`
npTest`-(${a} * ${a}).sum(axis=0, keepdims=True)`
npTest`-${a} * ${a}.sum(axis=0, keepdims=True)`
npTest`-(${a} * ${a}).sum(axis=-1, keepdims=True)`
npTest`np.sum(np.ones(np.array([10,2])), axis=0, keepdims=True) + np.arange(2)`

npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],None,:].shape`

npTest`np.arange(120).reshape([2,3,4,5])[:,0,[1,2],None,:]`


[ [ [ [Array] ], [ [Array] ] ], [ [ [Array] ], [ [Array] ] ] ]

In [ ]:
// $$.html("<svg><rect width=250 height=25 style='fill: orange;'></svg>")

In [16]:
var document = new JSDOM("").window.document;
var x = np`linspace(0,1,100)`
var y = np`${x} * ${x}`


var d3 = require('d3'); // Must be version 6.7.0 or less (install d3^6.7.0). In v7, they dropped support to commonJS's "require"
var { JSDOM } = require('jsdom');


// set the dimensions and margins of the graph
var margin = { top: 10, right: 30, bottom: 30, left: 60 },
  width = 460 - margin.left - margin.right,
  height = 400 - margin.top - margin.bottom;

// append the svg object to the body of the page
var svg = d3.select(document.body)
  .append("svg")
  .attr("width", width + margin.left + margin.right)
  .attr("height", height + margin.top + margin.bottom)
  .append("g")
  .attr("transform", "translate(" + margin.left + "," + margin.top + ")");

// Add X axis
var axisX = d3.scaleLinear()
  .domain([0, 1])
  .range([0, width]);
svg.append("g")
  .attr("transform", "translate(0," + height + ")")
  .call(d3.axisBottom(axisX));

// Add Y axis
var axisY = d3.scaleLinear()
  .domain([0, 1])
  .range([height, 0]);
svg.append("g")
  .call(d3.axisLeft(axisY));

// Add dots
svg.append('g')
  .selectAll("dot")
  .data(x.map((_,i)=>[x[i], y[i]]))
  .enter()
  .append("circle")
  .attr("cx", function (d) { return axisX(d[0]); })
  .attr("cy", function (d) { return axisY(d[1]); })
  .attr("r", 1.5)
  .style("fill", "#69b3a2");


$$.html(document.body.innerHTML);

0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0 0.0 0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.0